In [70]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import xlrd
pd.options.display.max_rows = 50
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

matplotlib.rcParams['pdf.fonttype'] = 42

%matplotlib inline


In [71]:
df = pd.read_csv('NYPD_Motor_Vehicle_Collisions.csv')

In [72]:
df.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,07/09/2019,0:00,BRONX,10454,40.811430,-73.924410,"(40.81143, -73.92441)",NaN,NaN,320 EAST 140 STREET,...,Unspecified,NaN,NaN,NaN,4167931,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1,07/09/2019,0:00,BROOKLYN,11203,40.643980,-73.937096,"(40.64398, -73.937096)",NaN,NaN,4209 CLARENDON ROAD,...,NaN,NaN,NaN,NaN,4168035,Sedan,NaN,NaN,NaN,NaN
2,07/09/2019,0:00,BROOKLYN,11215,40.677414,-73.983050,"(40.677414, -73.98305)",4 AVENUE,UNION STREET,NaN,...,Unspecified,NaN,NaN,NaN,4166913,Bus,Bus,NaN,NaN,NaN
3,07/09/2019,0:00,BROOKLYN,11236,40.652576,-73.915950,"(40.652576, -73.91595)",AVENUE A,EAST 93 STREET,NaN,...,Unspecified,Unspecified,NaN,NaN,4167437,Sedan,Sedan,Sedan,NaN,NaN
4,07/09/2019,0:00,QUEENS,11373,40.741420,-73.884670,"(40.74142, -73.88467)",NaN,NaN,80-09 45 AVENUE,...,Unspecified,NaN,NaN,NaN,4167245,Sedan,NaN,NaN,NaN,NaN


In [73]:
df.columns

Index(['DATE', 'TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE',
       'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
       'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
       'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
       'CONTRIBUTING FACTOR VEHICLE 5', 'UNIQUE KEY', 'VEHICLE TYPE CODE 1',
       'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4',
       'VEHICLE TYPE CODE 5'],
      dtype='object')

In [74]:
df = df[(df['VEHICLE TYPE CODE 1'] == 'Bike') | 
   (df['VEHICLE TYPE CODE 2'] == 'Bike') |
   (df['VEHICLE TYPE CODE 3'] == 'Bike') |
   (df['VEHICLE TYPE CODE 4'] == 'Bike') |
   (df['VEHICLE TYPE CODE 5'] == 'Bike')]



In [75]:
df.shape

(8186, 29)

In [76]:
df.dtypes

DATE                              object
TIME                              object
BOROUGH                           object
ZIP CODE                          object
LATITUDE                         float64
LONGITUDE                        float64
LOCATION                          object
ON STREET NAME                    object
CROSS STREET NAME                 object
OFF STREET NAME                   object
NUMBER OF PERSONS INJURED        float64
NUMBER OF PERSONS KILLED         float64
NUMBER OF PEDESTRIANS INJURED      int64
NUMBER OF PEDESTRIANS KILLED       int64
NUMBER OF CYCLIST INJURED          int64
NUMBER OF CYCLIST KILLED           int64
NUMBER OF MOTORIST INJURED         int64
NUMBER OF MOTORIST KILLED          int64
CONTRIBUTING FACTOR VEHICLE 1     object
CONTRIBUTING FACTOR VEHICLE 2     object
CONTRIBUTING FACTOR VEHICLE 3     object
CONTRIBUTING FACTOR VEHICLE 4     object
CONTRIBUTING FACTOR VEHICLE 5     object
UNIQUE KEY                         int64
VEHICLE TYPE COD

In [77]:
df['VEHICLE TYPE CODE 1'].value_counts()

Bike                                   3207
Sedan                                  2301
Station Wagon/Sport Utility Vehicle    1768
Taxi                                    518
Pick-up Truck                           109
Bus                                      64
Box Truck                                56
Van                                      36
Convertible                              17
4 dr sedan                               17
Motorcycle                               13
PK                                       10
Garbage or Refuse                         9
Tractor Truck Diesel                      8
Ambulance                                 5
Motorscooter                              4
Tanker                                    3
Flat Bed                                  3
Dump                                      3
Tow Truck / Wrecker                       2
van                                       2
Flat Rack                                 2
Motorbike                       

In [78]:
df.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
35,07/09/2019,10:00,QUEENS,11368,40.743230,-73.85492,"(40.74323, -73.85492)",NaN,NaN,52-10 108 STREET,...,Unspecified,NaN,NaN,NaN,4166842,Pick-up Truck,Bike,NaN,NaN,NaN
41,07/09/2019,10:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67 EAST DRIVE,...,NaN,NaN,NaN,NaN,4167398,Bike,NaN,NaN,NaN,NaN
52,07/09/2019,10:35,BROOKLYN,11213,40.665916,-73.92547,"(40.665916, -73.92547)",ROCKAWAY PARKWAY,EAST NEW YORK AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4166890,Sedan,Bike,NaN,NaN,NaN
128,07/09/2019,13:00,BROOKLYN,11226,40.650387,-73.95872,"(40.650387, -73.95872)",CHURCH AVENUE,FLATBUSH AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4167003,Van,Bike,NaN,NaN,NaN
198,07/09/2019,15:00,BRONX,10452,40.840424,-73.92239,"(40.840424, -73.92239)",WEST 170 STREET,SHAKESPEARE AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4167620,Sedan,Bike,NaN,NaN,NaN


In [79]:
df.columns

Index(['DATE', 'TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE',
       'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
       'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
       'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
       'CONTRIBUTING FACTOR VEHICLE 5', 'UNIQUE KEY', 'VEHICLE TYPE CODE 1',
       'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4',
       'VEHICLE TYPE CODE 5'],
      dtype='object')

In [80]:
df = df.drop(columns = ['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
       'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
       'CONTRIBUTING FACTOR VEHICLE 5' ])

In [81]:
df.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
35,07/09/2019,10:00,QUEENS,11368,40.743230,-73.85492,"(40.74323, -73.85492)",NaN,NaN,52-10 108 STREET,...,1,0,0,0,4166842,Pick-up Truck,Bike,NaN,NaN,NaN
41,07/09/2019,10:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67 EAST DRIVE,...,1,0,0,0,4167398,Bike,NaN,NaN,NaN,NaN
52,07/09/2019,10:35,BROOKLYN,11213,40.665916,-73.92547,"(40.665916, -73.92547)",ROCKAWAY PARKWAY,EAST NEW YORK AVENUE,NaN,...,1,0,0,0,4166890,Sedan,Bike,NaN,NaN,NaN
128,07/09/2019,13:00,BROOKLYN,11226,40.650387,-73.95872,"(40.650387, -73.95872)",CHURCH AVENUE,FLATBUSH AVENUE,NaN,...,1,0,0,0,4167003,Van,Bike,NaN,NaN,NaN
198,07/09/2019,15:00,BRONX,10452,40.840424,-73.92239,"(40.840424, -73.92239)",WEST 170 STREET,SHAKESPEARE AVENUE,NaN,...,1,0,0,0,4167620,Sedan,Bike,NaN,NaN,NaN


In [82]:
pat = r'.*(\d{4}\b)'
df.groupby(df['DATE'].str.extract(pat, expand = False))['NUMBER OF CYCLIST KILLED'].sum()

DATE
2012     0
2013     0
2014     0
2016     0
2017     1
2018     7
2019    15
Name: NUMBER OF CYCLIST KILLED, dtype: int64

In [83]:
df.groupby(df['DATE'].str.extract(pat, expand = False))['NUMBER OF CYCLIST INJURED'].sum()

DATE
2012       1
2013       1
2014       1
2016     216
2017      65
2018    3693
2019    2223
Name: NUMBER OF CYCLIST INJURED, dtype: int64

In [84]:
df.groupby([df['DATE'].str.extract(pat, expand = False),df['BOROUGH']])['NUMBER OF CYCLIST INJURED'].sum()

DATE  BOROUGH      
2012  MANHATTAN           1
2013  MANHATTAN           1
2014  MANHATTAN           1
2016  BRONX               8
      BROOKLYN           51
      MANHATTAN          61
      QUEENS             11
      STATEN ISLAND       5
2017  BRONX               5
      BROOKLYN           16
      MANHATTAN          11
      QUEENS              6
2018  BRONX             297
      BROOKLYN         1078
      MANHATTAN         718
      QUEENS            498
      STATEN ISLAND      31
2019  BRONX             163
      BROOKLYN          593
      MANHATTAN         492
      QUEENS            332
      STATEN ISLAND      17
Name: NUMBER OF CYCLIST INJURED, dtype: int64

In [85]:
df.groupby(df['DATE'].str.extract(pat, expand = False))['BOROUGH'].value_counts()

DATE  BOROUGH      
2012  MANHATTAN           1
2013  MANHATTAN           1
2014  MANHATTAN           1
2016  MANHATTAN         132
      BROOKLYN           80
      QUEENS             21
      BRONX              14
      STATEN ISLAND       7
2017  BROOKLYN           19
      MANHATTAN          19
      QUEENS             10
      BRONX               5
2018  BROOKLYN         1319
      MANHATTAN        1096
      QUEENS            575
      BRONX             382
      STATEN ISLAND      38
2019  BROOKLYN          748
      MANHATTAN         723
      QUEENS            395
      BRONX             216
      STATEN ISLAND      21
Name: BOROUGH, dtype: int64

In [86]:
df.groupby([df['DATE'].str.extract(pat, expand = False),df['BOROUGH']])['ON STREET NAME'].value_counts()

DATE  BOROUGH        ON STREET NAME                  
2012  MANHATTAN      CANAL STREET                        1
2014  MANHATTAN      EAST 34 STREET                      1
2016  BRONX          BEAUMONT AVENUE                     1
                     BERGEN AVENUE                       1
                     BROADWAY                            1
                     EAST 149 STREET                     1
                     EAST 193 STREET                     1
                     MORRIS PARK AVENUE                  1
                     MORRISON AVENUE                     1
                     SAINT ANNS AVENUE                   1
                     SOUTHERN BOULEVARD                  1
                     WESTCHESTER AVENUE                  1
      BROOKLYN       86 STREET                           3
                     5 AVENUE                            2
                     BEDFORD AVENUE                      2
                     EASTERN PARKWAY                     2
  

In [87]:
df.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
35,07/09/2019,10:00,QUEENS,11368,40.743230,-73.85492,"(40.74323, -73.85492)",NaN,NaN,52-10 108 STREET,...,1,0,0,0,4166842,Pick-up Truck,Bike,NaN,NaN,NaN
41,07/09/2019,10:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67 EAST DRIVE,...,1,0,0,0,4167398,Bike,NaN,NaN,NaN,NaN
52,07/09/2019,10:35,BROOKLYN,11213,40.665916,-73.92547,"(40.665916, -73.92547)",ROCKAWAY PARKWAY,EAST NEW YORK AVENUE,NaN,...,1,0,0,0,4166890,Sedan,Bike,NaN,NaN,NaN
128,07/09/2019,13:00,BROOKLYN,11226,40.650387,-73.95872,"(40.650387, -73.95872)",CHURCH AVENUE,FLATBUSH AVENUE,NaN,...,1,0,0,0,4167003,Van,Bike,NaN,NaN,NaN
198,07/09/2019,15:00,BRONX,10452,40.840424,-73.92239,"(40.840424, -73.92239)",WEST 170 STREET,SHAKESPEARE AVENUE,NaN,...,1,0,0,0,4167620,Sedan,Bike,NaN,NaN,NaN


In [88]:
df.groupby([df['DATE'].str.extract(pat, expand = False),df['BOROUGH']])['ON STREET NAME'].value_counts()

DATE  BOROUGH        ON STREET NAME                  
2012  MANHATTAN      CANAL STREET                        1
2014  MANHATTAN      EAST 34 STREET                      1
2016  BRONX          BEAUMONT AVENUE                     1
                     BERGEN AVENUE                       1
                     BROADWAY                            1
                     EAST 149 STREET                     1
                     EAST 193 STREET                     1
                     MORRIS PARK AVENUE                  1
                     MORRISON AVENUE                     1
                     SAINT ANNS AVENUE                   1
                     SOUTHERN BOULEVARD                  1
                     WESTCHESTER AVENUE                  1
      BROOKLYN       86 STREET                           3
                     5 AVENUE                            2
                     BEDFORD AVENUE                      2
                     EASTERN PARKWAY                     2
  

In [96]:
df[df['VEHICLE TYPE CODE 1']== 'Bike'].groupby([df['DATE'].str.extract(pat, expand = False),df['BOROUGH']])['NUMBER OF PEDESTRIANS INJURED'].sum()

DATE  BOROUGH      
2016  BRONX             4
      BROOKLYN         21
      MANHATTAN        57
      QUEENS           11
      STATEN ISLAND     1
2017  BRONX             0
      BROOKLYN          0
      MANHATTAN         1
      QUEENS            1
2018  BRONX             8
      BROOKLYN         43
      MANHATTAN        69
      QUEENS           13
      STATEN ISLAND     0
2019  BRONX            11
      BROOKLYN         23
      MANHATTAN        42
      QUEENS           11
      STATEN ISLAND     1
Name: NUMBER OF PEDESTRIANS INJURED, dtype: int64